# Pipeline

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

dataset = pd.read_csv('../Data/SMSSpamCollection.tsv', sep='\t', names=['label', 'message'])

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

dataset['message_length'] = dataset['message'].apply(lambda x: len(x) - x.count(" "))
dataset['punct%'] = dataset['message'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(dataset['message'])

X_features = pd.concat([dataset['message_length'], dataset['punct%'], pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names_out())], axis=1)
X_features.head()

,message_length,punct%,,0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,...,zindgi,zoe,zogtoriu,zoom,zouk,zyada,é,ü,üll,〨ud
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Build Grid Search with Cross Validation

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_features, dataset['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,40.657554,0.645346,0.306712,0.042039,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979354,0.974865,0.974865,0.969452,0.970350,0.973777,0.003575,1
7,22.109813,0.812827,0.361985,0.056069,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.975763,0.973968,0.969452,0.971249,0.973598,0.002940,2
3,2.874965,0.554448,0.171986,0.021025,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.978456,0.974865,0.978456,0.966757,0.969452,0.973597,0.004750,3
8,55.067059,0.301177,1.036516,0.080044,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.976661,0.973968,0.967655,0.970350,0.973418,0.003970,4
10,31.983598,4.508703,0.488462,0.197938,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.973968,0.973968,0.968553,0.971249,0.972880,0.002758,5
